In [1]:
import geopandas as gpd
import os
import pandas as pd
import openrouteservice 
os.chdir("/home/silas/projects/msc_thesis")

In [2]:
#set Api key for ORS
api_key= "5b3ce3597851110001cf624865e19fb4d0c2400e9aba8877785f6853"

#import flat dataset
flats_zh= gpd.read_file(r'./data/raw_data/geodata_stadt_Zuerich/building_stats/data/ssz.gwr_stzh_wohnungen.shp')

#import rcp dataset
rcps = gpd.read_file(r'./data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')


#filter out demolished flats
flats_zh_existing=flats_zh.query('wstatlang=="Bestehend"')
flats_zh_existing['egid']=flats_zh_existing['egid'].astype(int)

# check if CRS are matching
print("Current CRS:", flats_zh_existing.crs)
print("Current CRS:", rcps.crs)


Current CRS: epsg:2056
Current CRS: epsg:2056


/home/silas/miniconda3/envs/rasterio_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [3]:
# Create a subset of n flats
n = 100
buffer_distance = 600
flats_subset = flats_zh_existing.iloc[1:n,].copy()

# Create buffer
flats_subset['buffer'] = flats_subset.geometry.buffer(buffer_distance)

def find_points_in_buffer(buffer_geom):
    return rcps[rcps.geometry.within(buffer_geom)]

# Apply function to each buffer, save in dataframe
results = pd.DataFrame(columns=['flat_id', 'rcp'])
for idx, row in flats_subset.iterrows():
    points_in_buffer = find_points_in_buffer(row['buffer'])
    if not points_in_buffer.empty:
        # For each point in the buffer, add a new row
        for point_idx in points_in_buffer.index:
            results.loc[len(results)] = {
                'flat_id': row['egid'],
                'rcp': point_idx
            }

In [4]:
#function to get coordinates for rcp x
def get_rcp_coordinates(rcp_id):
   return rcps.geometry[rcp_id]

#function to get coordinates for each flat_id
def get_flat_coordinates(flat_id):
   return flats_subset.loc[flats_subset['egid']==flat_id, 'geometry']

In [ ]:
#convert to WGS84
flats_subset = flats_subset.to_crs(epsg=4326)
rcps = rcps.to_crs(epsg=4326)

1     1246779.539
89    1246779.539
90    1246779.539
dtype: float64
1247020.01


In [6]:

# Create ors client using the API key
client = openrouteservice.Client(key=api_key)

# Function to calculate the walking route between a flat and a recycling point
def calculate_walking_route(flat_id, rcp_id):
    flat_coords = get_flat_coordinates(flat_id).values[0]
    rcp_coords = get_rcp_coordinates(rcp_id)
    
    # Get the coordinates in the format required by OpenRouteService
    coords = ([flat_coords.x, flat_coords.y], [rcp_coords.x, rcp_coords.y])
    
    # Request the route from OpenRouteService
    route = client.directions(
        coordinates=coords,
        profile='foot-walking',
        format='geojson'
    )
    
    # Extract distance and duration
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    
    return distance, duration

# Apply the function to each row in the results dataframe
results['distance'] = 0.0
results['duration'] = 0.0

for idx, row in results.iterrows():
    distance, duration = calculate_walking_route(row['flat_id'], row['rcp'])
    results.at[idx, 'distance'] = distance
    results.at[idx, 'duration'] = duration/60 # Convert seconds to minutes

print(results.head())

/home/silas/miniconda3/envs/rasterio_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/rasterio_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/rasterio_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/rasterio_env/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} t

   flat_id  rcp  distance  duration
0   145253   47     813.5  9.761667
1   145253   48     517.9  6.213333
2   145253  129     613.7  7.365000
3   145253  130     324.6  3.895000
4   145253  131     276.8  3.321667


In [7]:
results[results['duration'] > 10]



,flat_id,rcp,distance,duration


In [8]:
results

,flat_id,rcp,distance,duration
0,145253,47,813.5,9.761667
1,145253,48,517.9,6.213333
2,145253,129,613.7,7.365000
3,145253,130,324.6,3.895000
4,145253,131,276.8,3.321667
...,...,...,...,...
484,145254,131,270.2,3.241667
485,145254,48,525.1,6.301667
486,145254,129,600.8,7.210000
487,145254,130,311.7,3.740000
